In [1]:
!pip install -U finance-datareader
!pip install opendartreader

In [2]:
import OpenDartReader
import FinanceDataReader as fdr
import pandas as pd
import time
from pathlib import Path

#api_key = '85c2367fdad752f376b3947ccdf9998092765f0e'  # OpenDart API KEY 장재영
api_key = '92d9f8aa9b71e2a04bb68d1ff79843860c2ed22a'  # OpenDart API KEY 강동표

In [3]:
import datetime

In [4]:
dart = OpenDartReader(api_key)

In [5]:
symbols = fdr.StockListing("KRX")
symbols = list(symbols['Symbol'])

In [6]:
symbols[:5]

['060310', '095570', '006840', '054620', '265520']

In [7]:
from tqdm import tqdm

In [ ]:
today = datetime.datetime.now().strftime("%Y-%m-%d")

In [ ]:
print('krx_statements-' + today + '.csv')

krx_statements-2022-02-25.csv


In [8]:
# '11013' = 1분기보고서, '11012' = 반기보고서, '11014' = 3분기보고서, '11011' = 사업보고서
reprt_code = ['11013', '11012', '11014', '11011']

In [9]:
finance_statements = pd.DataFrame(columns=['종목코드', '유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익',
                            '당기순이익', '영업활동현금흐름', '잉여현금흐름'], index=['1900-01-01'])

In [ ]:
#finance_statements.to_csv('/content/drive/MyDrive/22캡스톤디자인/krx_statements-' + today + '.csv')

In [ ]:
pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/krx_statements-2022-02-24.csv')

In [15]:
worked = []
for symbol in symbols:
  if str.isdigit(symbol[:5]) and len(symbol) < 7:
    worked.append(symbol)

In [ ]:
# 데이터를 얻기위한 반복문 시작

for symbol in tqdm(worked[203:]):
    print(symbol)
    all_finance_statements = pd.DataFrame(columns=['종목코드', '유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익',
                            '당기순이익', '영업활동현금흐름', '잉여현금흐름'], index=['1900-01-01'])
    # 데이터 초기화. 정리된 데이터를 모으는 곳이며 얻고자 하는 데이터 이름을
    # column 화 시킴.  빈 index 를 넣으면 추가가 안되므로 임으로 하나 넣자.

    # 정리 완료된 파일 저장하기 위한 경로 및 이름. result_종목이름.xlsx 형태로 저장된다.
    fileName = f'2022_1_KRX.csv'
    symbol = str(symbol)
    finance_statement = pd.DataFrame(columns=['종목코드', '유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익',
                            '당기순이익', '영업활동현금흐름', '잉여현금흐름'], index=['1900-01-01'])

    for i in range(2015, 2022):  # OpenDart는 2015년부터 정보를 제공한다.

        # 더미 리스트 초기화. 1 ~ 4 분기 데이터를 합할 예정이므로 4 크기 만큼의 리스트 선언.
        current_assets = [0, 0, 0, 0]  # 유동자산
        liabilities = [0, 0, 0, 0]  # 부채총계
        equity = [0, 0, 0, 0]  # 자본총계
        revenue = [0, 0, 0, 0]  # 매출액
        gross_marin = [0, 0, 0, 0]  # 매출총이익 #grossMargin
        EBIT = [0, 0, 0, 0]  # 영업이익 #EBIT
        net_income = [0, 0, 0, 0]  # 당기순이익
        cfo = [0, 0, 0, 0]  # 영업활동현금흐름
        cfi = [0, 0, 0, 0]  # 투자활동현금흐름
        fcf = [0, 0, 0, 0]  # 잉여현금흐름 : 편의상 영업활동 - 투자활동 현금흐름으로 계산

        for j, k in enumerate(reprt_code):
            df1 = pd.DataFrame()  # Raw Data
            df1 = df1.replace('', 0, regex=True)

            # 타입이 NoneType 이 아니면 읽어온다.
            temp = dart.finstate_all(symbol, i, reprt_code=k, fs_div='CFS')
            if str(type(temp)) != "<class 'NoneType'>":
                df1 = df1.append(temp)
                # 재무상태표 부분
                condition_1 = (df1.sj_nm == '재무상태표') & (df1.account_nm == '유동자산')  # 유동자산
                condition_2 = (df1.sj_nm == '재무상태표') & (df1.account_nm == '부채총계')  # 부채총계
                condition_3 = (df1.sj_nm == '재무상태표') & ((df1.account_nm == '자본총계') | (df1.account_nm == '반기말자본') | (
                        df1.account_nm == '3분기말자본') | (df1.account_nm == '분기말자본') | (
                                                                df1.account_nm == '1분기말자본'))  #자본총계
                # 손익계산서 부분
                condition_4 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '매출액') | (df1.account_nm == '수익(매출액)'))
                condition_5 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (df1.account_nm == '매출총이익')
                condition_6 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '영업이익(손실)') | (df1.account_nm == '영업이익'))
                condition_7 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '당기순이익(손실)') | (df1.account_nm == '당기순이익') |
                        (df1.account_nm == '분기순이익') | (df1.account_nm == '분기순이익(손실)') | (
                                df1.account_nm == '반기순이익') | (df1.account_nm == '반기순이익(손실)') |
                        (df1.account_nm == '연결분기순이익') | (df1.account_nm == '연결반기순이익') | (
                                df1.account_nm == '연결당기순이익') | (df1.account_nm == '연결분기(당기)순이익') | (
                                df1.account_nm == '연결반기(당기)순이익') |
                        (df1.account_nm == '연결분기순이익(손실)'))
                # 현금흐름표 부분
                condition_8 = (df1.sj_nm == '현금흐름표') & (
                        (df1.account_nm == '영업활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))

                condition_9 = (df1.sj_nm == '현금흐름표') & (
                        (df1.account_nm == '투자활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))

                conditions = [condition_1, condition_2, condition_3, condition_4, condition_5, condition_6, condition_7,
                              condition_8]

                stock_code = str(symbol)
                current_assets[j] = 0
                liabilities[j] = 0
                equity[j] = 0
                revenue[j] = 0
                gross_marin[j] = 0
                EBIT[j] = 0
                net_income[j] = 0
                cfo[j] = 0
                cfi[j] = 0

                #for condition in conditions:
                 #   if len(df1[condition]) != 0:
                  #      print(1)


                if len(df1[condition_1]) != 0:
                    current_assets[j] = int(df1.loc[condition_1].iloc[0]['thstrm_amount'])
                if len(df1[condition_2]) != 0:
                    liabilities[j] = int(df1.loc[condition_2].iloc[0]['thstrm_amount'])
                if len(df1[condition_3]) != 0:
                    equity[j] = int(df1.loc[condition_3].iloc[0]['thstrm_amount'])
                if len(df1[condition_4]) != 0:
                    revenue[j] = int(df1.loc[condition_4].iloc[0]['thstrm_amount'])
                if len(df1[condition_5]) != 0:
                    gross_marin[j] = int(df1.loc[condition_5].iloc[0]['thstrm_amount'])
                if len(df1[condition_6]) != 0:
                    EBIT[j] = int(df1.loc[condition_6].iloc[0]['thstrm_amount'])
                if len(df1[condition_7]) != 0:
                    net_income[j] = int(df1.loc[condition_7].iloc[0]['thstrm_amount'])
                if len(df1[condition_8]) != 0:
                    cfo[j] = int(df1[condition_8].iloc[0]['thstrm_amount'])
                if len(df1[condition_9]) != 0:
                    cfi[j] = int(df1[condition_9].iloc[0]['thstrm_amount'])

                fcf[j] = (cfo[j] - cfi[j])

                if k == '11013':  # 1분기.
                    path_string = str(i) + '-03-31'
                elif k == '11012':  # 2분기
                    path_string = str(i) + '-06-30'
                elif k == '11014':  # 3분기
                    path_string = str(i) + '-09-30'
                else:  # 4분기. 1 ~ 3분기 데이터를 더한다음 사업보고서에서 빼야 함
                    path_string = str(i) + '-12-30'
                    revenue[j] = revenue[j] - (revenue[0] + revenue[1] + revenue[2])
                    gross_marin[j] = gross_marin[j] - (gross_marin[0] + gross_marin[1] + gross_marin[2])
                    EBIT[j] = EBIT[j] - (EBIT[0] + EBIT[1] + EBIT[2])
                    net_income[j] = net_income[j] - (net_income[0] + net_income[1] + net_income[2])
                    fcf[j] = fcf[j] - (fcf[0] + fcf[1] + fcf[2])

                # 데이터프레임에 저장하는 부분
                finance_statement.loc[path_string] = [str(stock_code), current_assets[j], liabilities[j], equity[j],
                                        revenue[j], gross_marin[j], EBIT[j], net_income[j], cfo[j], fcf[j]]

                finance_statement.tail()  # 데이터프레임 끝에 저장한다.
                # 너무 잦은 요청이 있을 경우 OpenDart API 측에서 IP 를 차단하니 텀을 둔다.
                time.sleep(0.1)
            else:
                pass
            finance_statements.to_csv
    all_finance_statements = pd.concat([all_finance_statements, finance_statement[1:]],)
    all_finance_statements.to_csv('/content/drive/MyDrive/22캡스톤디자인/krx_statements-2022-02-25.csv', mode='a', header=False, index=True)

    # 원본 dataframe 에도 영향을 끼치게끔 (inplace=True) 첫 행 drop.
    # df2.drop(['1900-01-01'], inplace=True) # 첫 행 drop

# 파일 저장. fileName 에 명시된 경로에 각 종목코드별로 다른 이름으로 저장
#all_finance_statements.to_csv('/content/drive/MyDrive/22캡스톤디자인/finance_statements.csv')

  0%|          | 0/3232 [00:00<?, ?it/s]

024840


  0%|          | 1/3232 [00:49<44:15:07, 49.31s/it]

148020


  0%|          | 2/3232 [01:18<33:38:27, 37.49s/it]

285000


# 40 + 11 + 54 + 30 + 45 + 6 + 58 + 34 + 17 + 40 + 336 + 440



In [12]:
# 처음부터 부터
now = 40 + 11 + 54 + 30 + 45 + 6 + 58 + 34 + 17 + 40 + 336 + 440 + 310
print(f"앞에서부터 {now}")
print(f"{7782 - now} 남음")

앞에서부터 1421
6361 남음


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/krx_statements-2022-02-24.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431806 entries, 0 to 431805
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  431806 non-null  object 
 1   종목코드        431406 non-null  float64
 2   유동자산        431406 non-null  float64
 3   부채총계        431406 non-null  float64
 4   자본총계        431406 non-null  float64
 5   매출액         431406 non-null  float64
 6   매출총이익       431406 non-null  float64
 7   영업이익        431406 non-null  float64
 8   당기순이익       431406 non-null  float64
 9   영업활동현금흐름    431406 non-null  float64
 10  잉여현금흐름      431406 non-null  float64
dtypes: float64(10), object(1)
memory usage: 36.2+ MB


In [ ]:
df = df.drop_duplicates(['Unnamed: 0','종목코드'])

In [ ]:
df.head(30)

,Unnamed: 0,종목코드,유동자산,부채총계,자본총계,매출액,매출총이익,영업이익,당기순이익,영업활동현금흐름,잉여현금흐름
0,1900-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-12-30,60310.0,1.606908e+10,2.942016e+10,3.793981e+10,2.321054e+10,1.635011e+09,-4.157989e+09,-5.599033e+09,0.000000e+00,0.000000e+00
3,2016-06-30,60310.0,1.358261e+10,2.138256e+10,3.488834e+10,6.534630e+09,1.646555e+08,-1.222254e+09,-1.840069e+09,0.000000e+00,0.000000e+00
4,2016-09-30,60310.0,1.798534e+10,2.467833e+10,3.506178e+10,7.357351e+09,6.386867e+07,-7.660802e+08,-4.897230e+08,0.000000e+00,0.000000e+00
5,2016-12-30,60310.0,1.342667e+10,2.314263e+10,3.460376e+10,9.850441e+09,3.690977e+09,3.164870e+09,-1.073646e+09,0.000000e+00,0.000000e+00
6,2017-06-30,60310.0,1.296385e+10,2.217000e+10,2.785500e+10,7.192916e+09,2.456186e+08,0.000000e+00,0.000000e+00,2.717232e+09,2.717232e+09
7,2017-09-30,60310.0,1.542777e+10,2.388559e+10,2.820808e+10,9.225760e+09,1.955538e+09,1.171768e+09,3.862790e+08,4.812661e+09,4.812661e+09
8,2017-12-30,60310.0,1.671287e+10,2.452724e+10,2.917967e+10,1.006761e+10,-8.928945e+07,-3.592993e+09,-9.190089e+09,0.000000e+00,-7.529894e+09
9,2018-03-31,60310.0,1.199212e+10,1.687057e+10,3.146390e+10,7.536829e+09,0.000000e+00,-1.225498e+09,-1.173541e+09,-1.103669e+09,-1.008845e+09
10,2018-06-30,60310.0,1.329354e+10,1.825814e+10,3.096567e+10,6.743874e+09,7.172636e+08,-2.274215e+08,-4.627946e+08,-2.019487e+08,-2.190692e+07


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 0 to 121213
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1472 non-null   object 
 1   종목코드        1471 non-null   float64
 2   유동자산        1471 non-null   float64
 3   부채총계        1471 non-null   float64
 4   자본총계        1471 non-null   float64
 5   매출액         1471 non-null   float64
 6   매출총이익       1471 non-null   float64
 7   영업이익        1471 non-null   float64
 8   당기순이익       1471 non-null   float64
 9   영업활동현금흐름    1471 non-null   float64
 10  잉여현금흐름      1471 non-null   float64
dtypes: float64(10), object(1)
memory usage: 138.0+ KB


```javascript
function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60 * 1000)
```

In [10]:
worked = []
for symbol in symbols:
  if str.isdigit(symbol[:5]) and len(symbol) < 7:
    worked.append(symbol)

In [11]:
len(worked)

3435

In [13]:
worked.index('024840')

203

In [14]:
worked[203:]

['024840',
 '148020',
 '285000',
 '361580',
 '287300',
 '287310',
 '290080',
 '284980',
 '287320',
 '287330',
 '252400',
 '252410',
 '252420',
 '284990',
 '285010',
 '285020',
 '315480',
 '105780',
 '290130',
 '368200',
 '367760',
 '367770',
 '388280',
 '326240',
 '385560',
 '385550',
 '385540',
 '270800',
 '292050',
 '307010',
 '319870',
 '403990',
 '234310',
 '241390',
 '401170',
 '300640',
 '266160',
 '114100',
 '282000',
 '295000',
 '295020',
 '397420',
 '397410',
 '276650',
 '375270',
 '411720',
 '417450',
 '399580',
 '336160',
 '326230',
 '272560',
 '196230',
 '315960',
 '252730',
 '252720',
 '379780',
 '219390',
 '354240',
 '368590',
 '267440',
 '267490',
 '267450',
 '267500',
 '388420',
 '140570',
 '140580',
 '379790',
 '183710',
 '310080',
 '174360',
 '136340',
 '281990',
 '272570',
 '250730',
 '291680',
 '371150',
 '183700',
 '270810',
 '278240',
 '275750',
 '302450',
 '361590',
 '334690',
 '334700',
 '253280',
 '253290',
 '105560',
 '024120',
 '002380',
 '021320',
 '344820',